In [1]:
from keras.applications import MobileNet

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D

from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model

In [2]:
img_rows, img_cols = 224, 224

MobileNet = MobileNet(weights = 'imagenet',
                     include_top = False,
                     input_shape = (img_rows, img_cols, 3))

for layer in MobileNet.layers:
    layer.trainable = False
    
for (i, layer) in enumerate(MobileNet.layers):
    print(str(i),layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 ReLU False
4 DepthwiseConv2D False
5 BatchNormalization False
6 ReLU False
7 Conv2D False
8 BatchNormalization False
9 ReLU False
10 ZeroPadding2D False
11 DepthwiseConv2D False
12 BatchNormalization False
13 ReLU False
14 Conv2D False
15 BatchNormalization False
16 ReLU False
17 DepthwiseConv2D False
18 BatchNormalization False
19 ReLU False
20 Conv2D False
21 BatchNormalization False
22 ReLU False
23 ZeroPadding2D False
24 DepthwiseConv2D False
25 BatchNormalization False
26 ReLU False
27 Conv2D False
28 BatchNormalization False
29 ReLU False
30 DepthwiseConv2D False
31 BatchNormalization False
32 ReLU False
33 Conv2D False
34 BatchNormalization False
35 ReLU False
36 ZeroPadding2D False
37 DepthwiseConv2D False
38 BatchNormalization False
39 ReLU False
40 Conv2D False
41 BatchNormalization False
42 ReLU False
43 DepthwiseConv2D False
44 BatchNormalization False
45 ReLU False
46 Conv2D False
47 BatchNormalization False
48

In [3]:
def addTopModelMobileNet(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(512, activation = 'relu')(top_model)
    top_model = Dense(num_classes, activation = 'softmax')(top_model)
    return top_model

In [4]:
num_classes = 10

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [5]:
train_data_dir = './Data/train/'
val_data_dir = './Data/validation/'

train_datagen = ImageDataGenerator(rescale = 1.0 / 255,
                                   rotation_range = 45,
                                   width_shift_range = .3,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest')

val_datagen = ImageDataGenerator(rescale = 1.0 / 255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_rows, img_cols),
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(val_data_dir,
                                                       target_size=(img_rows, img_cols),
                                                       batch_size = batch_size,
                                                       class_mode = 'categorical')

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [8]:
checkpoint = ModelCheckpoint('./Model/monkey_breed_MobileNet.h5',
                             monitor = 'loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'loss',
                         min_delta= 0,
                         patience = 3,
                         verbose=1,
                         restore_best_weights = True)

callbacks = [earlystop, checkpoint]

model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = .001),
              metrics = ['accuracy'])

no_train = 1098
no_val = 272

In [11]:
epochs = 5
batch_size = 32

history = model.fit(train_generator,
                    steps_per_epoch = no_train // batch_size,
                    epochs = epochs,
                    callbacks = callbacks,
                    validation_data = validation_generator,
                    validation_steps = no_val // batch_size)

Epoch 1/5
34/34 [==============================] - 44s 1s/step - loss: 0.3848 - accuracy: 0.8743 - val_loss: 0.1640 - val_accuracy: 0.9453

Epoch 00001: loss improved from 0.51300 to 0.38482, saving model to ./Model\monkey_breed_MobileNet.h5
Epoch 2/5
34/34 [==============================] - 50s 1s/step - loss: 0.3615 - accuracy: 0.9184 - val_loss: 0.1809 - val_accuracy: 0.9414

Epoch 00002: loss improved from 0.38482 to 0.36151, saving model to ./Model\monkey_breed_MobileNet.h5
Epoch 3/5
34/34 [==============================] - 52s 2s/step - loss: 0.4162 - accuracy: 0.9062 - val_loss: 0.2585 - val_accuracy: 0.9297

Epoch 00003: loss did not improve from 0.36151
Epoch 4/5
34/34 [==============================] - 48s 1s/step - loss: 0.2373 - accuracy: 0.9447 - val_loss: 0.2342 - val_accuracy: 0.9375

Epoch 00004: loss improved from 0.36151 to 0.23730, saving model to ./Model\monkey_breed_MobileNet.h5
Epoch 5/5
34/34 [==============================] - 49s 1s/step - loss: 0.3519 - accurac